## Création première Database

In [ ]:
import psycopg2
from psycopg2 import sql
import pandas as pd


df = pd.read_csv("../data/strava_activities_clean_.csv")

host = "localhost"
port = "5432"
dbname = "postgres"
user = "arthurdercq"
password = "qsdfgqsdfg"

try:
    conn = psycopg2.connect(
        host=host,
        port=port,
        dbname=dbname,
        user=user,
        password=password
    )
    print("Connexion réussie !")

    # Créer un curseur pour exécuter des requêtes
    cur = conn.cursor()
    cur.execute("SELECT version();")
    print("Version de PostgreSQL :", cur.fetchone())

    # Fermer le curseur
    cur.close()

except Exception as e:
    print("Erreur de connexion :", e)


Connexion réussie !
Version de PostgreSQL : ('PostgreSQL 17.5 (Homebrew) on aarch64-apple-darwin23.6.0, compiled by Apple clang version 16.0.0 (clang-1600.0.26.6), 64-bit',)


In [2]:
def store_df_in_postgresql(db_path, host, database, user, password, port):

    # Lire le fichier CSV dans un DataFrame pandas
    df = pd.read_csv(db_path)

    # Connexion à la base de données PostgreSQL
    # Remplacez les valeurs par vos informations de connexion PostgreSQL
    conn = psycopg2.connect(
        host = host,
        database = database,
        user = user,
        password= password,
        port = port
    )


    # Créer un curseur pour exécuter des commandes SQL
    cur = conn.cursor()

    # Nom de la table dans laquelle vous allez importer les données
    table_name = "activities"

    # Créer la table dans PostgreSQL
    # Assurez-vous que les types de données correspondent à ceux de votre CSV
    create_table_query = sql.SQL("""
    CREATE TABLE IF NOT EXISTS {} (
    id SERIAL PRIMARY KEY,
    name VARCHAR(255),
    distance FLOAT,
    moving_time INTEGER,
    elapsed_time INTEGER,
    total_elevation_gain FLOAT,
    sport_type VARCHAR(255),
    start_date TIMESTAMP,
    start_date_local TIMESTAMP,
    timezone VARCHAR(50),
    achievement_count INTEGER,
    kudos_count INTEGER,
    gear_id VARCHAR(255),
    start_latlng VARCHAR(50),
    end_latlng VARCHAR(50),
    average_speed FLOAT,
    max_speed FLOAT,
    average_cadence FLOAT,
    average_temp FLOAT,
    has_heartrate BOOLEAN,
    average_heartrate FLOAT,
    max_heartrate FLOAT,
    elev_high FLOAT,
    elev_low FLOAT,
    pr_count INTEGER,
    has_kudoed BOOLEAN,
    average_watts FLOAT,
    kilojoules FLOAT
);
""").format(sql.Identifier(table_name))

    cur.execute(create_table_query)

    # Insérer les données dans la table
    for index, row in df.iterrows():
        insert_query = sql.SQL("""
        INSERT INTO {} (
            name, distance, moving_time, elapsed_time, total_elevation_gain,
            sport_type, start_date, start_date_local, timezone, achievement_count,
            kudos_count, gear_id, start_latlng, end_latlng, average_speed,
            max_speed, average_cadence, average_temp, has_heartrate,
            average_heartrate, max_heartrate, elev_high, elev_low, pr_count,
            has_kudoed, average_watts, kilojoules
        ) VALUES (
        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
    )
        """).format(sql.Identifier(table_name))

        cur.execute(insert_query, (

        row['name'], row['distance'], row['moving_time'], row['elapsed_time'],
        row['total_elevation_gain'], row['sport_type'], row['start_date'],
        row['start_date_local'], row['timezone'], row['achievement_count'],
        row['kudos_count'], row['gear_id'], str(row['start_latlng']),
        str(row['end_latlng']), row['average_speed'], row['max_speed'],
        row['average_cadence'], row['average_temp'], row['has_heartrate'],
        row['average_heartrate'], row['max_heartrate'], row['elev_high'],
        row['elev_low'], row['pr_count'], row['has_kudoed'],
        row['average_watts'], row['kilojoules']
    ))

    # Valider les changements
    conn.commit()

    # Fermer le curseur et la connexion
    cur.close()
    conn.close()

    print("Données importées dans PostgreSQL ✅")

In [3]:
host = "localhost"
database = "postgres"
user = "arthurdercq"
password = "qsdfgqsdfg"
port = "5432"


store_df_in_postgresql(db_path="../data/strava_activities_clean_.csv", host=host, database=database, user=user, password=password, port=port)

Données importées dans PostgreSQL ✅


## Mise à jour de la Database

In [4]:
import sys
import os

# Ajoute le chemin absolu du dossier projet au PATH
sys.path.append(os.path.abspath("/Users/arthurdercq/code/Data Science/Garmin_Dashboard"))


In [ ]:
!pip install sqlalchemy psycopg2-binary

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 1.1 MB/s eta 0:00:00m eta 0:00:010:01:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 1.6 MB/s eta 0:00:003.2 MB/s eta 0:00:01
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.4.0
    Uninstalling typing_extensions-4.4.0:
      Successfully uninstalled typing_extensions-4.4.0

[notice] A new release of pip is available: 23.1.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
from sqlalchemy import create_engine
from eye_sight.strava.fetch_strava import update_strava_data
from eye_sight.strava.clean_data import clean_data


# 1 Charger les nouvelles données STRAVA
new_data = update_strava_data()
new_data_clean = clean_data(new_data)

# 2 Connexion à la base PostgreSQL locale
engine = create_engine(f"postgresql://:<{user}><{password}>@{host}:{port}/<{database}>")

# Étape 1 : Récupérer les 10 derniers IDs présents dans la base
with engine.connect() as conn:
    result = conn.execute("SELECT id FROM activities ORDER BY id DESC LIMIT 10;")
    existing_ids = [row[0] for row in result]


# Étape 3 : Filtrer uniquement les nouvelles activités
filtered_data = new_data[~new_data['id'].isin(existing_ids)]

# Étape 4 : Insérer dans la base
if not filtered_data.empty:
    filtered_data.to_sql('activities', engine, if_exists='append', index=False)
    print(f"{len(filtered_data)} nouvelles activités ajoutées.")
else:
    print("Aucune nouvelle activité à ajouter.")